### HKU Practical Bioinformatics 2022-2023 - "NGS sequence alignment":how we alignment the NGS sequence data

*by Dr Jason Wong*

The objective of this course is to show demonstrate how sequencing data of human is aligned to the reference genome sequence and to study their effect on life growth and disease.


# create a linux terminal

In [ ]:
!pip install colab-xterm
%load_ext colabxterm
%xterm

## *** Set working directory

By default working directoy will be My Drive/PB_course

In [ ]:
# set working pathway to your own google drive doc (~ 1 min)
from google.colab import drive
drive.mount('/content/gdrive')                         # if using for the first time, you be requested to grant permission to link your Google Drive

import os
try:
  os.mkdir("/content/gdrive/My Drive/PB_course")         # change this path if necessary
except FileExistsError:
  print("directory already exist. OK to continue")
os.chdir("/content/gdrive/My Drive/PB_course")

In [ ]:
!pwd

## *** Package installation and downloads for workshop (~ 5 minutes)

1.   conda (for simple installation of packages)
2.   FastQC (for reads quality check)
3.   bwa (tools for sequence alignment)
4.   samtools (tools for processing sam & bam files)
5.   Download ready prepared files for analysis.

In [ ]:
# install conda (~ 1 min). There will be a message saying that the session has crashed but don't worry about this. This is due to the session restarting following conda installation
!pip install -q condacolab
import condacolab#
condacolab.install()

In [ ]:
# install fastqc (~ 2 mins)
!conda install -c bioconda fastqc

In [ ]:
# install bwa (~ 1 mins)
!conda install -c bioconda bwa

In [ ]:
# install samtools (~1 mins)
!conda install -c bioconda samtools

In [ ]:
#download reference sequence
# double check that we are in right directory (~ 30s)
import os
os.chdir("/content/gdrive/MyDrive/PB_course")                     # change this path if necessary

import os
if os.path.isfile("/content/gdrive/MyDrive/PB_course/DB_trunc/chr2.fa"):    # check if the file exist
  print("reference file already exit, OK to continue.") 
else:
  !wget -O DB_trunc.zip https://cloudstor.aarnet.edu.au/plus/s/2ogImYtpLaiqkS4/download  # download if file not exist
  #unzip fasta file
  !unzip DB_trunc.zip
  !rm DB_trunc.zip

!ls -l ./DB_trunc/

In [ ]:
#download sample sequences
import os
os.chdir("/content/gdrive/My Drive/PB_course/")
if os.path.isfile("/content/gdrive/MyDrive/PB_course/Datasets/ChIP-seq_H3K27ac_example.fq.gz"):    # check if the file exist
  print("file already exit, OK to continue.") 
else:
 !wget -O Datasets.zip https://github.com/jasonwong-lab/HKU-Practical-Bioinformatics/raw/main/files/Datasets.zip    # download necessary file
 !unzip -o Datasets.zip   #unzip file
 !rm Datasets.zip

## NGS_alignment command line

1.1 Quality control

1.2. Burrows–Wheeler Aligner

1.3. Working with SAM files

1.4. Align the WGS paired-end file

In [ ]:
#Check what datasets we have downloaded
%cd /content/gdrive/MyDrive/PB_course
!ls -l

%cd Datasets/
!ls -l

In [ ]:
# To look at the ChIP-seq fastq file type (head -n 12 to print first 12 lines):
!zcat < ChIP-seq_H3K27ac_example.fq.gz |head -n 12

In [ ]:
# How many reads are there in the file? Type your code below:


### Quality control

In [ ]:
## Run FastQC
!fastqc ChIP-seq_H3K27ac_example.fq.gz

In [ ]:
# check result html file 
!ls

#download file and check html file with your local browser
from google.colab import files
files.download('ChIP-seq_H3K27ac_example_fastqc.html')

you can also download the file **ChIP-seq_H3K27ac_example_fastqc.html** from "gdrive/MyDrive/PB_course/Datasets" on the left side

### Burrows–Wheeler Aligner

In [ ]:
# Look at the options for bwa and bwa mem
!bwa

In [ ]:
!bwa mem

In [ ]:
# let's take a look at the database files
%cd /content/gdrive/My Drive/PB_course/DB_trunc

!ls -l

In [ ]:
#Get ready to run BWA. First Go into Datasets directory
%cd /content/gdrive/My Drive/PB_course/Datasets

In [ ]:
# make an directory to store output file
!mkdir BAM

In [ ]:
#Do sequence alignment
!bwa mem ../DB_trunc/chr2.fa ./ChIP-seq_H3K27ac_example.fq.gz > ./BAM/ChIP-seq_H3K27ac_example.sam

In [ ]:
# check the result
!head -n 20 ./BAM/ChIP-seq_H3K27ac_example.sam

### Working with SAM files

In [ ]:
#Check the samtools command
!samtools

In [ ]:
#Check out some stats about our aligned file
!samtools flagstat ./BAM/ChIP-seq_H3K27ac_example.sam

In [ ]:
#Prepare the file for viewing on genome browser:
#Step 1 – Convert SAM to BAM
!samtools view -b ./BAM/ChIP-seq_H3K27ac_example.sam > ./BAM/ChIP-seq_H3K27ac_example.bam

In [ ]:
#Step 2 – sort BAM file
!samtools sort ./BAM/ChIP-seq_H3K27ac_example.bam > ./BAM/ChIP-seq_H3K27ac_example_sorted.bam

In [ ]:
#Step 3 – index BAM file
!samtools index ./BAM/ChIP-seq_H3K27ac_example_sorted.bam

In [ ]:
#Look at the files that we have created
!ls -l ./BAM/

### IGV borwser

In [ ]:
#Install igv-notebook
!pip install igv-notebook

In [ ]:
#Load track from local paths
import igv_notebook

igv_notebook.init()

b = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr2:47,782,081-47,807,953"
    }
)

b.load_track(
    {
        "name": "ChIP-seq_H3K27ac",
        "path": "./BAM/ChIP-seq_H3K27ac_example_sorted.bam",
        "indexPath": "./BAM/ChIP-seq_H3K27ac_example_sorted.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

### Align the WGS paired-end file

In [ ]:
# Check that we have the WGS fastq files
%cd /content/gdrive/My Drive/PB_course/Datasets
!ls -l

In [ ]:
# If you don't have WGS_example_1.fq.gz or WGS_example_2.fq.gz, run this cell to redownload the files
import os
os.chdir("/content/gdrive/My Drive/PB_course")

import os
if os.path.isfile("/content/gdrive/MyDrive/PB_course/Datasets/WGS_example_1.fq.gz"):    # check if the file exist
  print("reference file already exit, OK to continue.") 
else:
 !wget -O Datasets.zip https://github.com/jasonwong-lab/HKU-Practical-Bioinformatics/raw/main/files/Datasets.zip
 !unzip -o Datasets.zip   #unzip file
 !rm Datasets.zip

#current directory should still be ~/Datasets
%cd Datasets/
!ls -l

In [ ]:
#look at our work directory and paired-end file:
!zcat < WGS_example_1.fq.gz |head -1
!zcat < WGS_example_2.fq.gz |head -1

In [ ]:
#align the WGS paired-end file:( ~ 1 mins)
#Step 1 – Aligning paired-end file using bwa mem:
!bwa mem ../DB_trunc/chr2.fa ./WGS_example_1.fq.gz ./WGS_example_2.fq.gz > ./BAM/WGS_example.sam


In [ ]:
#Step 2 – Output sorted BAM, this time use piping to skip one step:
!samtools view -b ./BAM/WGS_example.sam | samtools sort >./BAM/WGS_example_sorted.bam

In [ ]:
#Step 3 – Index sorted bam file
!samtools index ./BAM/WGS_example_sorted.bam

In [ ]:
#Check stats about the aligned WGS file
!samtools flagstat ./BAM/WGS_example_sorted.bam

In [ ]:
#IGV
#Load track from local paths
import igv_notebook

igv_notebook.init()

b = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr2:47,782,081-47,807,953"
    }
)

b.load_track(
    {
        "name": "ChIP-seq H3K27ac",
        "path": "./BAM/ChIP-seq_H3K27ac_example_sorted.bam",
        "indexPath": "./BAM/ChIP-seq_H3K27ac_example_sorted.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

b.load_track(
    {
        "name": "WGS",
        "path": "./BAM/WGS_example_sorted.bam",
        "indexPath": "./BAM/WGS_example_sorted.bam.bai",
        "format": "bam",
        "type": "alignment"
    })

### self-exercise

There are two more datasets in the Datasets folder. We will be using them next week so it will be good to get them aligned today or before next week’s lecture.

They are WXS_example_*.fq.gz and WXS_example_cancer_*.fq.gz

In [ ]:
# Run this cell to download the WXS files
import os
os.chdir("/content/gdrive/My Drive/PB_course")

import os
if os.path.isfile("/content/gdrive/MyDrive/PB_course/Datasets/WXS_example_cancer_1.fq.gz"):    # check if the file exist
  print("reference file already exit, OK to continue.") 
else:
 !wget -O Datasets_GXS.zip https://github.com/jasonwong-lab/HKU-Practical-Bioinformatics/raw/main/files/Datasets_GXS.zip
 !unzip -o Datasets_GXS.zip   #unzip file
 !rm Datasets_GXS.zip
 !wget -O Datasets/WXS_example_cancer_1.fq.gz https://github.com/jasonwong-lab/HKU-Practical-Bioinformatics/raw/main/files/WXS_example_cancer_1.fq.gz
 !wget -O Datasets/WXS_example_cancer_2.fq.gz https://github.com/jasonwong-lab/HKU-Practical-Bioinformatics/raw/main/files/WXS_example_cancer_2.fq.gz
 

#Check what files we have now
%cd Datasets/
!ls -l

In [ ]:
#type you code here....

